In [47]:
import pandas as pd
import numpy as np
import nltk
import spacy
nltk.download('stopwords')
from tqdm import tqdm
from text_to_num import text2num
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem.lancaster import LancasterStemmer
tqdm.pandas(desc="my bar!")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/matheusmbr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
# try:
#     nlp = spacy.load('pt_core_news_lg')
# except:
#     import os
#     os.system('python -m spacy download pt_core_news_lg')
#     nlp = spacy.load('pt_core_news_lg')

In [49]:
year = 2013
df = pd.read_csv(f'./Raw/{year}.csv', sep=';', encoding ='latin')
df_processed = df[['DS_RESUMO', 'DS_PALAVRA_CHAVE', 'DS_ABSTRACT', 'DS_KEYWORD', 'NM_AREA_AVALIACAO']].copy()

In [50]:
df_processed.head()

,DS_RESUMO,DS_PALAVRA_CHAVE,DS_ABSTRACT,DS_KEYWORD,NM_AREA_AVALIACAO
0,A POSSIBILIDADE DA OCORRÊNCIA DE SISMOS NO BRA...,"ANÁLISE SÍSMICA, CONCRETO ARMADO, DETALHAMENTO...",NÃO INFORMADO,NÃO INFORMADO,ENGENHARIAS I
1,AS OBRAS DE PONTES ESTAIADAS ENVOLVEM UMA TECN...,PONTE ESTAIADA. PONTE DO SABER. ANÁLISE ESTRUT...,NÃO INFORMADO,NÃO INFORMADO,ENGENHARIAS I
2,A HIPERPLASIA DE CÉLULAS NEUROENDÓCRINAS DO LA...,1. DOENÇAS PULMONARES INTERSTICIAIS  RADIOGRA...,NEUROENDOCRINE CELL HYPERPLASIA OF INFANCY (NE...,NÃO INFORMADO,MEDICINA II
3,ESTUDOS RECENTES TEM DEMONSTRADO ALTERAÇÕES DA...,1. NEURALGIA DO TRIGÊMEO - FISIOPATOLOGIA. 2. ...,RECENT STUDIES HAVE DETECTED LOCAL CHANGES OF ...,NÃO INFORMADO,MEDICINA II
4,DENTRE AS TÉCNICAS MAIS UTILIZADAS NA DETERMIN...,"PLATAFORMAS FIXAS, ESTRUTURAS OFFSHORE, ESTRUT...",NÃO INFORMADO,NÃO INFORMADO,ENGENHARIAS I


In [51]:
df_processed.columns = df_processed.columns.str.slice(3).str.lower()
df_processed.rename(columns={
    'base': 'ano'
}, inplace=True)

for item in list(df_processed.iloc[:, 0:].columns):
    df_processed[f'{item}'] = df_processed[f'{item}'].str.lower()

In [52]:
df_processed.isna().sum()

resumo            0
palavra_chave     0
abstract          7
keyword           0
area_avaliacao    0
dtype: int64

In [53]:
df_processed.dropna(subset=['abstract'], inplace=True)

df_processed.fillna({ 'projeto': 'não informado' }, inplace=True)

# df_processed.drop(index=df_processed[
#     df_processed.resumo.str.contains("não informado") | df_processed.palavra_chave.str.contains("não informado") | 
#     df_processed.abstract.str.contains("não informado") | df_processed.keyword.str.contains("não informado") | 
#     df_processed.grande_area_conhecimento.str.contains("não informado")
# ].index, inplace=True)
# df_processed.reset_index(drop=True, inplace=True)

In [54]:
idx = df_processed[(df_processed.palavra_chave == df_processed.keyword) | (df_processed.resumo == df_processed.abstract) | (df_processed.palavra_chave == df_processed.resumo) | (df_processed.keyword == df_processed.abstract)].index
df_processed.drop(index=idx, inplace=True)
df_processed.reset_index(drop=True, inplace=True)
del idx

In [55]:
def rename_areas(text: str) -> str:
    if text in ['administração pública e de empresas, ciências contábeis e turismo', 'administração, ciências contábeis e turismo']:
        return 'administração pública e de empresas, ciências contábeis e turismo'
    elif text in ['arquitetura e urbanismo', 'arquitetura, urbanismo e design']:
        return 'arquitetura, urbanismo e design'
    elif text in 'filosofia/teologia:subcomissão filosofia':
        return 'filosofia'
    elif text in ['filosofia/teologia:subcomissão teologia', 'teologia']:
        return 'ciências da religião e teologia'
    elif text in 'letras / linguística':
        return 'linguística e literatura'
    elif text in 'artes / música':
        return 'artes'
    elif text in 'ciências sociais aplicadas i':
        return 'serviço social'
    else:
        return text

In [56]:
df_processed.area_avaliacao = df_processed.area_avaliacao.progress_apply(rename_areas)
del rename_areas

my bar!: 100%|██████████| 43077/43077 [00:00<00:00, 1175983.03it/s]


In [57]:
colegios_dict = {
    'ciências da vida': {
        'ciências agrárias': ['ciência de alimentos', 'ciências agrárias i', 'medicina veterinária', 'zootecnia / recursos pesqueiros'],
        'ciências biológicas': ['biodiversidade', 'ciências biológicas i', 'ciências biológicas ii', 'ciências biológicas iii'],
        'ciências da saúde': ['educação física', 'enfermagem', 'farmácia', 'medicina i', 'medicina ii', 'medicina iii', 'nutrição', 'odontologia', 'saúde coletiva']
    },
    'humanidades': {
        'ciências humanas': ['antropologia / arqueologia', 'ciência política e relações internacionais', 'ciências da religião e teologia', 'educação', 'filosofia', 'geografia', 'história', 'psicologia', 'sociologia'],
        'ciências sociais aplicadas': ['administração pública e de empresas, ciências contábeis e turismo', 'arquitetura, urbanismo e design', 'comunicação e informação', 'direito', 'economia', 'planejamento urbano e regional / demografia', 'serviço social'],
        'linguística, letras e artes': ['artes', 'linguística e literatura']
    },
    'ciências exatas, tecnológicas e multidisciplinar': {
        'ciências exatas e da terra': ['astronomia / física', 'ciência da computação', 'geociências', 'matemática / probabilidade e estatística', 'química'],
        'engenharias': ['engenharias i', 'engenharias ii', 'engenharias iii', 'engenharias iv'],
        'multidisciplinar': ['biotecnologia', 'ciências ambientais', 'ensino', 'interdisciplinar', 'materiais']
    }
}

In [58]:
df_processed['colegios'] = ''
df_processed['areas'] = ''
df_processed.rename(columns={'area_avaliacao': 'subareas'}, inplace=True)

for colegios, areas in colegios_dict.items():
    for areas, subareas in areas.items():
        df_processed.loc[df_processed.subareas.isin(subareas), 'colegios'] = colegios
        df_processed.loc[df_processed.subareas.isin(subareas), 'areas'] = areas
        
df_processed = df_processed[['resumo', 'palavra_chave', 'abstract', 'keyword', 'colegios', 'areas', 'subareas']]

In [59]:
df_processed.head()

,resumo,palavra_chave,abstract,keyword,colegios,areas,subareas
0,a hiperplasia de células neuroendócrinas do la...,1. doenças pulmonares intersticiais  radiogra...,neuroendocrine cell hyperplasia of infancy (ne...,não informado,ciências da vida,ciências da saúde,medicina ii
1,estudos recentes tem demonstrado alterações da...,1. neuralgia do trigêmeo - fisiopatologia. 2. ...,recent studies have detected local changes of ...,não informado,ciências da vida,ciências da saúde,medicina ii
2,nesse trabalho objetivamos testar a performanc...,1. espectroscopia de ressonância magnética. 2....,the aim of this study was to test the performa...,não informado,ciências da vida,ciências da saúde,medicina ii
3,"a qualidade de vida, enquanto orientadora de p...",qualidade de vida; espaço urbano; parquet; aca...,"the quality of life, while guiding by public p...",quality of life. urban places. parks. outdoor ...,"ciências exatas, tecnológicas e multidisciplinar",multidisciplinar,interdisciplinar
4,a síndrome de sjögren (ss) é uma doença autoim...,1. glândula parótida - anatomia & histologia. ...,sjögren syndrome (ss) is a common auto-immune ...,não informado,ciências da vida,ciências da saúde,medicina ii


In [60]:
def remove_number_words(text: str) -> str:
    converted = []
    for word in wordpunct_tokenize(text):
        try:
            int(text2num(word, lang='pt'))
        except:
            try:
                int(text2num(word, lang='en'))
            except:
                converted.append(word)

    return ' '.join([str(word) for word in converted])

In [61]:
df = df_processed.copy()

In [62]:
for item in ['palavra_chave', 'keyword']:
    df_processed[f'{item}'].replace(to_replace ='[2-9][.]', value = ';', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='[1][.]', value = '', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='\[(pt|en)\]', value = ';', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='\,', value = ';', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='[/]+', value = ';', regex = True, inplace=True)
    df_processed[f'{item}'].replace('\x96', '-', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace =r'[^a-z0-9\s,.:;?!-]', value = '', regex = True)
    df_processed[f'{item}'].replace(to_replace=r'(\w{2,})\.', value='\\1;', regex=True)
    df_processed[f'{item}'].replace(to_replace ='[ ][-][ ]', value = ';', regex = True, inplace=True)
    # df_processed[f'{item}'].replace(to_replace ='[^\w;]+', value = '', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='[;][ ][;]', value = ';', regex = True, inplace=True)
    df_processed[f'{item}'] = df_processed[f'{item}'].str.lstrip(' ')
    df_processed[f'{item}'] = df_processed[f'{item}'].str.lstrip(';')

In [63]:
test = df_processed.palavra_chave.copy()

In [64]:
test = list(set(';'.join(test).split(';')))
test = [item for item in test if len(item) > 1]

for i in range (len(test)):
    test[i] = test[i].strip()

In [65]:
aux = df_processed[df_processed.palavra_chave.str.contains(' c ')].palavra_chave

In [66]:
[item for item in test if len(item) < 2]

['1',
 'd',
 '4',
 'p',
 '5',
 'n',
 't',
 'e',
 'f',
 '0',
 '7',
 'r',
 '2',
 'a',
 '3',
 'á',
 'b',
 '4',
 'c',
 'v',
 'h',
 'e',
 'z',
 'i',
 'k',
 'm']

In [ ]:
[item for item in test if 100 < len(item) < 300]

['desenvolvido na linha de pesquisa formação e profissionalização docente da faculdade de educação da universidade federal de goiás campus de goiânia',
 'ação cultural biblioteca biblioteca história biblioteca pública informação pública política cultural políticas públicas',
 'cana de açúcar ecologia microbiana genes mudança climática rizosfera rna ribossômico sistemas de cultivo',
 'recentemente vários trabalhos correlacionam perfis de expressão de mirnas com capacidade de resposta a diferentes tratamentos quimioterápicos cujos padrões de expressão estão associados a quimioresistencia ou sensibilidade ao tratamento',
 'historia econã mica â cade â polã tica industrial â economia polã ticaâ sistema brasileiro de defesa da concorrãªncia',
 'isótopos ferro oxigênio hidrogênio erosão dos solos reação de oxidação redução matéria orgânica homem',
 'criança desmame do respirador ensaio clínico controlado aleatório insuficiência respiratória respiração artificial unidade de terapia intensiva'

In [40]:
freq_colegio = dict()
freq_area = dict()
freq_subarea = dict()

for character in test:
    freq_colegio[character] = {}
    freq_area[character] = {}
    freq_subarea[character] = {}
    
    for item in sorted(df_processed.colegios.unique()):
        freq_colegio[character][item] = 0
        
    for item in sorted(df_processed.areas.unique()):
        freq_area[character][item] = 0
        
    for item in sorted(df_processed.subareas.unique()):
        freq_subarea[character][item] = 0
    
    corte = df_processed[df_processed.resumo.str.contains(character)]
        
    colegio = corte.colegios
    area = corte.areas
    subarea = corte.subareas
    
    for item in sorted(colegio.unique()):
        freq_colegio[character][item] = colegio.value_counts()[item]
        
    for item in sorted(area.unique()):
        freq_area[character][item] = area.value_counts()[item]
        
    for item in sorted(subarea.unique()):
        freq_subarea[character][item] = subarea.value_counts()[item]
        
freq_colegio = pd.DataFrame(freq_colegio).transpose()
freq_area = pd.DataFrame(freq_area).transpose()
freq_subarea = pd.DataFrame(freq_subarea).transpose()

In [41]:
freq_colegio.to_pickle('freq_colegio.pkl')
freq_area.to_pickle('freq_area.pkl')
freq_subarea.to_pickle('freq_subarea.pkl')

In [10]:
for item in ['resumo', 'abstract', 'palavra_chave', 'keyword']:
    df_processed[f'{item}'].replace(to_replace ='[0-9]+', value = '', regex = True, inplace=True)
    df_processed[f'{item}'] = df_processed[f'{item}'].progress_apply(remove_number_words)
    df_processed[f'{item}'].replace(to_replace ='[,.:;!?]+', value = '', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='[/<>()|\+\-\$%&#@\'\"]+', value = '', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='\W+', value = ' ', regex = True, inplace=True)

my bar!: 100%|██████████| 49852/49852 [00:03<00:00, 13585.94it/s]


In [11]:
for item in ['palavra_chave', 'keyword']:
    df_processed[f'{item}'].replace(to_replace ='([0-9][. ])', value = ',', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='([.])+', value = ',', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='(  )+', value = ' ', regex = True, inplace=True)
    df_processed[f'{item}'] = df_processed[f'{item}'].str.lstrip(', ')

In [12]:
df_processed.head()

,ano,subtipo_producao,projeto,resumo,palavra_chave,abstract,keyword,area_avaliacao
0,2013,dissertação,não informado,a possibilidade da ocorrência de sismos no bra...,análise sísmica concreto armado detalhamento s...,não informado,não informado,engenharias i
1,2013,dissertação,não informado,as obras de pontes estaiadas envolvem tecnolog...,ponte estaiada ponte do saber análise estrutur...,não informado,não informado,engenharias i
2,2013,tese,hiperplasia de células neuroendrócrinas em lac...,a hiperplasia de células neuroendócrinas do la...,doenças pulmonares intersticiais radiografia d...,neuroendocrine cell hyperplasia of infancy neh...,não informado,medicina ii
3,2013,tese,avaliação através das técnicas avançadas de rm...,estudos recentes tem demonstrado alterações da...,neuralgia do trigêmeo fisiopatologia neuralgia...,recent studies have detected local changes of ...,não informado,medicina ii
4,2013,dissertação,não informado,dentre as técnicas mais utilizadas na determin...,plataformas fixas estruturas offshore estrutur...,não informado,não informado,engenharias i


## Testes

In [17]:
df_test = df_processed[['resumo', 'area_avaliacao']].copy()

In [18]:
def remove_stopwords(text: str) -> str:
    stop_words = set(stopwords.words('portuguese'))
    new_text = [item for item in wordpunct_tokenize(text) if item not in stop_words]
    return " ".join(set(new_text))

In [19]:
df_test.resumo = df_test.resumo.apply(remove_stopwords)

In [20]:
def lc_stem(text: str) -> str:
    stemmer = LancasterStemmer()
    words = [stemmer.stem(word) for word in wordpunct_tokenize(text)]
    return " ".join(set(words))

In [21]:
lem = df_test.copy()
lem.resumo = lem.resumo.apply(lc_stem)

In [22]:
def spc_lem(text: str) -> str:
    words = [token.lemma_ for token in nlp(text) if token.pos_ != 'PUNCT']
    return " ".join(set(words))

In [23]:
stem = df_test.copy()
stem.resumo = stem.resumo.apply(spc_lem)

In [24]:
def freq_dist(array: list) -> dict:
    text = ' '.join(array)
    return FreqDist(wordpunct_tokenize(text))

In [28]:
lem.to_pickle(f'./Processed/{year}_lem.pkl')
stem.to_pickle(f'./Processed/{year}_stem.pkl')